In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.losses import Huber
from tensorflow.keras import regularizers
from time import time
from sklearn.metrics import mean_absolute_error
import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In my previous post, I went through the following process:

Environment: Anaconda, Windows 11.

- Data wrangling (Light exploration, followed by removing and transforming some variables)
- Split the training dataset into training and validation datasets
- Fit a model using a Scikit-Learn pipeline (Data Preprocessing + fitting XGBoost/LightGBM estimators with a Randomized Search across their respective hyperparameters)
- Evaluate and visualize model performance
- Implement an automated approach to selecting hyperparameters (HyperOpt)
- Make predictions

In this post, I will implement the following using the same wrangled/preprocessed data:

Environment: Docker, Windows Subsystem for Linux 2 (WSL 2), Windows 11.

- Build a simple Sequential model in Keras/Tensorflow
- Use the Weights and Biases (WandB) platform to select optimal hyperparameters and record experiments.
  - Experiments are evaluated using K-Fold Cross Validation. Mean RMSE across folds for each experiment are custom logged in WandB. 
- Make predictions.
- Blend predictions from the previous post (decision tree) and this post (neural net).
  - By taking the mean of predictions, taking the weighted mean of predictions, and defining a meta-model.
  - Predictions are evaluated on a holdout dataset, kept separate from the training set since the beginning of the project.

Actually setting up the environment (Docker, Windows Subsystem for Linux 2 (WSL 2), Windows 11, VSCode, using CUDA) took a couple of days and is probably worthy of its own blog post. It required the following process:

- Install WSL2, CUDA drivers and Docker
    - Define a Dockerfile that uses a base image compatible with CUDA
    - Get libraries from requirements.txt
    - Set "runArgs" in devcontainer.json to allow GPU usage
- Run in VSCode (the Jupyter extension gave me some trouble)
    - I ended up creating the container directly from a Dockerfile in the same repo as my code

In [3]:
# Read in data
# Function to bring in wrangled/preprocessed data from previous post
def data():
    training = pd.read_csv("../sklearn/training_preprocessed")
    validation = pd.read_csv("../sklearn/validation_preprocessed")
    holdout = pd.read_csv("../sklearn/holdout_preprocessed")
    holdout_predictions_df = pd.read_csv("../sklearn/holdout_preds_preprocessed")
    test = pd.read_csv("../sklearn/test_preprocessed")
    
    X_train = training.drop(columns="SalePrice")
    y_train = training["SalePrice"]
    X_valid = validation.drop(columns="SalePrice")
    y_valid = validation["SalePrice"]
    X_holdout = holdout.drop(columns="Actual_SalePrice")
    y_holdout = holdout["Actual_SalePrice"]
    X_test = test
    holdout_predictions_df = holdout_predictions_df
    return X_train, y_train, X_valid, y_valid, X_holdout, y_holdout, X_test, holdout_predictions_df

# Bring in data
X_train, y_train, X_valid, y_valid, X_holdout, y_holdout, X_test, holdout_predictions_df = data()

# Since this model uses k-fold validation, we don't need separate training and validation datasets
X_train = X_train.append(X_valid).reset_index().drop(columns="index")
y_train = y_train.append(y_valid).reset_index().drop(columns="index").values

/tmp/ipykernel_1783/2886453018.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X_train = X_train.append(X_valid).reset_index().drop(columns="index")
/tmp/ipykernel_1783/2886453018.py:24: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_train = y_train.append(y_valid).reset_index().drop(columns="index").values


In [4]:
# Log into Weights and Biases
wandb.init(project="house-price-prediction", entity="luiscostigan")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: luiscostigan (use `wandb login --relogin` to force relogin)


The Neural Network I will use for this task is a simple sequential model. It features:

- Two Dense layers with configurable dimensionality and L2 regularization.
- A configurable Dropout layer, which takes a sample of the data to improve generalizability of the model.
- Use of the ADAM optimizer with a configurable learning rate "scheduler" (since the learning rate will change over time).
- A configurable number of epochs and batch-size.

In [6]:
# Define simple Sequential model
def create_model():
    model = Sequential()
    model.add(Dense(wandb.config.dense1, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=regularizers.l2(wandb.config.bias1)))
    model.add(Dropout(wandb.config.dropout1))
    model.add(Dense(wandb.config.dense2, activation='relu', kernel_regularizer=regularizers.l2(wandb.config.bias2)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(wandb.config.learning_rate), loss="mean_absolute_error", metrics=[RootMeanSquaredError()])
    
    return model

The code below uses the Weights and Biases API to go through 15 "sweeps", with each sweep using a set of values for each configurable hyperparameter. Every sweep is conducted 5 times as part of a K-Fold cross validation process. The hyperparameter values to be evaluated for each sweep are determined using Bayesian Optimization (explained in my previous blog post). This requires some metric to optimize for, so I have used the mean RMSE across the 5 folds.

The Dense layers have fairly high dimensionality, since running a few trial experiments showed far better performance on the holdout dataset for overparameterized models.

In [7]:
# Define training function and hyperparameter ranges
from wandb.keras import WandbCallback
from sklearn.model_selection import KFold
# import multiprocessing

sweep_config = {
  "name": "keras-sequential-model-sweep",
  "method": "bayes",
  "metric": {
    "name": "Mean Validation RMSE (all folds)",
    "goal": "minimize"
  },
  "parameters": {
    "dropout1": {
      "distribution": "uniform",
      "min": 0.0,
      "max": 0.4
    },
    "dense1": {
      "distribution": "categorical",
      "values": [1024, 2048, 4096]
    },
    "dense2": {
      "distribution": "categorical",
      "values": [1024, 2048, 4096]
    },
    "bias1": {
      "distribution": "categorical",
      "values": [0.001, 0.01]
    },
    "bias2": {
      "distribution": "categorical",
      "values": [0.001, 0.01]
    },
    "epochs": {
      "distribution": "categorical",
      "values": [50, 100, 150]
    },
    "batch_size": {
      "distribution": "categorical",
      "values": [8, 16, 32]
    },
    "learning_rate": {
      "distribution": "categorical",
      "values": [0.0001, 0.001, 0.01]
    }
  }
}

config_defaults = {
  "dropout1": 0.1,
  "dense1": 2048,
  "dense2": 2048,
  "bias1": 0.01,
  "bias2": 0.01,
  "epochs": 100,
  "batch_size": 16,
  "learning_rate": 0.001
}

# Define number of splits
kf = KFold(n_splits=5)

def train():

  rmse_per_fold = []
  loss_per_fold = []
  fold_no = 1

  # Go through each split, and get the index number for each
  for train, test in kf.split(X_train, y_train):

    # With the current session in WandB
    with wandb.init(config=config_defaults) as run:

      def evaluate_model():

        # Recreate the model each time for each new batch
        model = None # Not sure if this step is necessary
        model = create_model()

        # Fit model on new batches
        model.fit(
        np.asarray(X_train),
        y_train, 
        epochs=wandb.config.epochs, 
        batch_size=wandb.config.batch_size, 
        verbose=0,
        callbacks=[WandbCallback()], 
        validation_data=(np.asarray(X_train),y_train)
        )
        
        # Generate data for each
        scores = model.evaluate(np.asarray(X_train), y_train, callbacks=[WandbCallback()])
        print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]}')
        rmse_per_fold.append(scores[1])
        loss_per_fold.append(scores[0])

        # == Provide average scores ==
        print('------------------------------------------------------------------------')
        print('Score per fold')
        for i in range(0, len(rmse_per_fold)):
            print('------------------------------------------------------------------------')
            print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - RMSE: {rmse_per_fold[i]}')
            print('------------------------------------------------------------------------')
        
        print('Average scores for all folds:')
        print(f'> RMSE: {np.mean(rmse_per_fold)} (+- {np.std(rmse_per_fold)})')
        print(f'> Loss: {np.mean(loss_per_fold)}')
        print('------------------------------------------------------------------------')


        wandb.log({
        "Mean Validation RMSE (all folds)": np.mean(rmse_per_fold),
        "Mean Validation Loss (all folds)": np.mean(loss_per_fold) 
        })

        wandb.join()

      evaluate_model()

      # TF has a problem releasing cache after an operation, for which the suggested solution is to start and kill a separate process (https://github.com/tensorflow/tensorflow/issues/36465)
      # However, attempting to do this results in an error with WandB (https://github.com/wandb/client/issues/1994)
      # p = multiprocessing.Process(target=evaluate_model) 
      # p.start() 
      # p.join()

      # Increase fold number
      fold_no = fold_no + 1

keras_sequential_sweep_1 = wandb.sweep(sweep_config, project="house-price-prediction", entity="luiscostigan")

count = 15

wandb.agent(keras_sequential_sweep_1, function=train, count=count)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: e8odal49
Sweep URL: https://wandb.ai/luiscostigan/house-price-prediction/sweeps/e8odal49


wandb: Agent Starting Run: wzk1y0kt with config:
wandb: 	batch_size: 8
wandb: 	bias1: 0.01
wandb: 	bias2: 0.001
wandb: 	dense1: 4096
wandb: 	dense2: 2048
wandb: 	dropout1: 0.26141804024777965
wandb: 	epochs: 100
wandb: 	learning_rate: 0.001


2022-03-14 02:31:04.126154: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-14 02:31:04.145074: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-14 02:31:04.145375: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-14 02:31:04.146346: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

37/37 [==============================] - 0s 4ms/step - loss: 0.2773 - root_mean_squared_error: 0.2948
Score for fold 1: loss of 0.27732130885124207; root_mean_squared_error of 0.29478657245635986
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.27732130885124207 - RMSE: 0.29478657245635986
------------------------------------------------------------------------
Average scores for all folds:
> RMSE: 0.29478657245635986 (+- 0.0)
> Loss: 0.27732130885124207
------------------------------------------------------------------------



...

Mean Validation Loss (all folds),▁
Mean Validation RMSE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▅▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▇▅▅▅▄▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
val_loss,█▃▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_root_mean_squared_error,█▃▄▂▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
Mean Validation Loss (all folds),0.12459
Mean Validation RMSE (all folds),0.16569
best_epoch,33
best_val_loss,0.10208


37/37 [==============================] - 0s 3ms/step - loss: 0.1254 - root_mean_squared_error: 0.1794
Score for fold 4: loss of 0.1254054754972458; root_mean_squared_error of 0.1793666034936905
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.1801561415195465 - RMSE: 0.2293308526277542
------------------------------------------------------------------------
------------------------------------------------------------------------
> Fold 2 - Loss: 0.08247493207454681 - RMSE: 0.11812463402748108
------------------------------------------------------------------------
------------------------------------------------------------------------
> Fold 3 - Loss: 0.11115256696939468 - RMSE: 0.14960092306137085
------------------------------------------------------------------------
------------------------------------------------------------------------
> Fold 4 - Los

Mean Validation Loss (all folds),▁
Mean Validation RMSE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▅▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▆▅▄▃▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▆▃▂▂▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁
val_root_mean_squared_error,███▃▃▂▂▁▁▁▁▂▁▂▁▂▁▁▁▂▂▁▁▂▂▁▂▂▂▂▂▂▂▂▄▁▂▁▂▁
Mean Validation Loss (all folds),0.1248
Mean Validation RMSE (all folds),0.16911
best_epoch,25
best_val_loss,0.09478


37/37 [==============================] - 0s 3ms/step - loss: 0.1407 - root_mean_squared_error: 0.1838
Score for fold 5: loss of 0.14069266617298126; root_mean_squared_error of 0.1838429570198059
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.1801561415195465 - RMSE: 0.2293308526277542
------------------------------------------------------------------------
------------------------------------------------------------------------
> Fold 2 - Loss: 0.08247493207454681 - RMSE: 0.11812463402748108
------------------------------------------------------------------------
------------------------------------------------------------------------
> Fold 3 - Loss: 0.11115256696939468 - RMSE: 0.14960092306137085
------------------------------------------------------------------------
------------------------------------------------------------------------
> Fold 4 - Lo

Mean Validation Loss (all folds),▁
Mean Validation RMSE (all folds),▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▅▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
root_mean_squared_error,█▇▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▂▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▁▁▁▁▁▁▁▂▁▂▁▂▁▁▁▁▁▁
val_root_mean_squared_error,█▄▃▄▁▁▁▁▁▁▁▁▁▁▂▁▁▁▃▂▂▂▁▁▁▁▁▂▁▂▁▂▁▂▂▁▁▁▁▁
Mean Validation Loss (all folds),0.12798
Mean Validation RMSE (all folds),0.17205
best_epoch,144
best_val_loss,0.09398


One of the main selling points of Weights and Biases is that is can track the results of experiments to make it easy for an ML practitioner to review previous experiments, including the hyperparamater values tested and their results on a validation set/during cross-validation. For this model I conducted 15 experiments, the results of which can be viewed in interactive graphs as below:

<img src="./wandb_graphs.png" width="800">

The set of hyperparameters resulting in the lowest loss (in terms of RMSE) is noted in the top row of the below image, again taken from Weights and Biases:

<img src="./wandb_rmse_loss.png" width="800">

In [8]:
# Enter best params from sweep
best_params = {
    "dropout": 0.001897,
    "dense1": 4096,
    "dense2": 2048,
    "bias1": 0.01,
    "bias2": 0.001,
    "epochs": 150,
    "batch_size": 8,
    "learning_rate": 0.001
}

# Build model using the best parameters
def make_predictions(best_params, dataset):
    
    model = Sequential()
    model.add(Dense(best_params.get("dense1"), input_dim=X_train.shape[1], activation='relu', kernel_regularizer=regularizers.l2(best_params.get("bias1"))))
    model.add(Dropout(best_params.get("dropout")))
    model.add(Dense(best_params.get("dense2"), activation='relu', kernel_regularizer=regularizers.l2(best_params.get("bias2"))))
    model.add(Dense(1))
    model.compile(optimizer=Adam(best_params.get("learning_rate")), loss="mean_absolute_error", metrics=[RootMeanSquaredError()])
    
    model.fit(X_train, y_train, epochs=best_params.get("epochs"), batch_size=best_params.get("batch_size"), verbose = 0)
                
    preds  = model.predict(dataset, best_params.get("batch_size"), verbose = 0)
              
    return preds

In [13]:
# Make predictions
test_predictions_log_transformed = make_predictions(best_params, X_test)

# Generate predictions on holdout dataset
holdout_predictions_log_transformed = make_predictions(best_params, X_holdout)

I'll first test my predictions from just the neural network in Kaggle before blending predictions. The result was reasonable.

<img src="./submission_nn_score.png" width="600">

In [15]:
# Generating the CSV for NN submission 
# Undo the log transform
test_predictions = np.exp(test_predictions_log_transformed)
holdout_predictions = np.exp(holdout_predictions_log_transformed)

d = {"Id":X_test.index,"SalePrice":test_predictions.flatten()}
submission = pd.DataFrame(data=d, index=None)

submission["Id"] = submission["Id"] + 1461

submission.to_csv("submission_nn.csv",index=False)

## Blending Predictions

So far, I have generated predictions using two decision tree-based models and a neural net-based model.
Before training and wrangling any data for either model, I isolated 10% of the training data set as a "holdout" set to evaluate different methods of blending predictions. I use three different methods to blend predictions:

1) Take the mean of predictions from XGBoost, LightGBM and the Neural Network model.
2) Take a weighted mean of all three, with weights based on how well each model performs on the holdout set.
3) Test a bunch of meta models and use cross-validation to determine the best.

In [65]:
from sklearn.metrics import mean_squared_error

# Refresh data
X_train, y_train, X_valid, y_valid, X_holdout, y_holdout, X_test, holdout_predictions_df = data()

# Append holdout set NN predictions to DT predictions
holdout_predictions_df["NN_Predictions"] = holdout_predictions_log_transformed
holdout_predictions_df = holdout_predictions_df[["XGBoost_Predictions", "LightGBM_Predictions", "NN_Predictions", "Actual_SalePrice"]]

# Add mean column
holdout_predictions_df["Mean_SalePrice"] = holdout_predictions_df[["XGBoost_Predictions", "LightGBM_Predictions", "NN_Predictions"]].mean(axis=1)

# Calculate RMSE
print("Unweighted Mean RMSE: " + str(mean_squared_error(holdout_predictions_df["Actual_SalePrice"], holdout_predictions_df["Mean_SalePrice"], squared=False)))

Unweighted Mean RMSE:0.12437362669476726


I submitted predictions based on the mean of all three to Kaggle, with the following results:

<img src="./submission_mean_score.png" width="600">

In [68]:
print("XGBoost RMSE: " + str(mean_squared_error(holdout_predictions_df["Actual_SalePrice"], holdout_predictions_df["XGBoost_Predictions"], squared=False)))
print("LightGBM RMSE: " + str(mean_squared_error(holdout_predictions_df["Actual_SalePrice"], holdout_predictions_df["LightGBM_Predictions"], squared=False)))
print("NN RMSE: " + str(mean_squared_error(holdout_predictions_df["Actual_SalePrice"], holdout_predictions_df["NN_Predictions"], squared=False)))

XGBoost RMSE: 0.13149106483301198
LightGBM RMSE: 0.15526025452567674
NN RMSE: 0.11821275132550024


The above shows the RMSE of each of the three models I implemented on the holdout set. While I have more confidence in the ability of models with lower loss to perform well with production data, the point of blending is to improve the generalizability of your predictions. Based on that, I decided to weight the predictions of each model accordingly:

30% - XGBoost
20% - LightGBM
50% - Neural Network

In [69]:
# Add weighted mean column
weight = [0.3, 0.2, 0.5]
holdout_predictions_df["Weighted_Mean_SalePrice"] = holdout_predictions_df[["XGBoost_Predictions", "LightGBM_Predictions", "NN_Predictions"]].dot(weight)

# Calculate RMSE
print("Weighted Mean RMSE:" + str(mean_squared_error(holdout_predictions_df["Actual_SalePrice"], holdout_predictions_df["Weighted_Mean_SalePrice"], squared=False)))

Weighted Mean RMSE:0.11975113619483989


The weighted mean predictions performed better than any individual model, and the unweighted mean. The score took me to 636 of 4,354 on the Kaggle leaderboard.

<img src="./submission_weighted_mean_score.png" width="600">

In [72]:
# Generate CSV for mean submission
# Get XGBoost and LightGBM predictions on test set from previous notebook
mean_predictions_df = pd.read_csv("../sklearn/xgb_lgb_test_predictions.csv")

# Reverse the log transform
mean_predictions_df[["XGBoost_Predictions","LightGBM_Predictions"]] = mean_predictions_df[["XGBoost_Predictions","LightGBM_Predictions"]].applymap(np.exp)

# Append NN Predictions
mean_predictions_df["NN_Predictions"] = test_predictions

# Take the mean
mean_predictions_df["SalePrice"] = mean_predictions_df[["XGBoost_Predictions", "LightGBM_Predictions", "NN_Predictions"]].mean(axis=1)

# Generate new df and save as CSV
mean_predictions_submission = mean_predictions_df[["Id","SalePrice"]]
mean_predictions_submission.to_csv("submission_mean.csv",index=False)

In [73]:
# Generate CSV for weighted mean submission
# Copy df
weighted_mean_predictions_df = mean_predictions_df

# Set weights and apply to predictions
weight = [0.3, 0.2, 0.5]
weighted_mean_predictions_df["SalePrice"] = weighted_mean_predictions_df[["XGBoost_Predictions", "LightGBM_Predictions", "NN_Predictions"]].dot(weight)

# Generate new df and save as CSV
weighted_mean_submission = weighted_mean_predictions_df[["Id","SalePrice"]]
weighted_mean_submission.to_csv("submission_weighted_mean.csv",index=False)

Next, I'll try defining a meta-model to best blend predictions from the three models. The meta-model was a simple grid search across different estimators, without attempting to optimize hyperparameters. Since the holdout dataset is very small, I used 10 folds in the cross-validation process.

In [79]:
import warnings
warnings.filterwarnings("ignore", message=".*Int64Index.*")

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
import xgboost as xgb
import lightgbm as lgb

# Defining a custom loss function (Root Mean Squared Error)
rmse = make_scorer(mean_squared_error, squared=False)

X = holdout_predictions_df[["XGBoost_Predictions", "LightGBM_Predictions", "NN_Predictions"]]
y = holdout_predictions_df["Actual_SalePrice"]

estimators = [
    {
        "clf": (LinearRegression(),)
    },
    {
        "clf": (Ridge(),)
    },
    {
        "clf": (xgb.XGBRegressor(),)
    },
    {
       "clf": (lgb.LGBMRegressor(),)
    }
]

pipe = Pipeline([("clf", LinearRegression())])

grid_search = GridSearchCV(pipe, estimators, cv=10, scoring=rmse)
grid_search.fit(X,y)
print("Holdout Set Cross-Validation RMSE Loss: " + str(grid_search.cv_results_.get("mean_test_score").mean()))

Holdout Set Cross-Validation RMSE Loss: 0.13143476657527364


The cross-validation RMSE seems within the same range as RMSE loss on the holdout dataset using the mean and weighted mean. However, when posting to Kaggle, performance was not as good as I'd hoped. I think there's a lot of scope for improving this, but that will have to wait for another day.

<img src="./submission_modelled_score.png" width="600">

In [41]:
# Generate CSV for modelled prediction submission
# Get test set predictions from all three models
modelled_predictions_df = mean_predictions_df
modelled_predictions_df[["XGBoost_Predictions", "LightGBM_Predictions", "NN_Predictions"]] = modelled_predictions_df[["XGBoost_Predictions", "LightGBM_Predictions", "NN_Predictions"]].applymap(np.log)

# Make predictions 
grid_search_blended_predictions_log_transformed = grid_search.predict(modelled_predictions_df[["XGBoost_Predictions", "LightGBM_Predictions", "NN_Predictions"]])
grid_search_blended_predictions = np.exp(grid_search_blended_predictions_log_transformed)

# Add predictions SalePrice column
modelled_predictions_df["SalePrice"] = grid_search_blended_predictions

# Generate new df and save as CSV
modelled_submission = modelled_predictions_df[["Id","SalePrice"]]
modelled_submission.to_csv("submission_modelled.csv",index=False)